# 01_scrap

## Getting data from ThaiPBS

In [1]:
from bs4 import BeautifulSoup as soup
import requests
import datetime
import pandas as pd
import json

In [2]:
thaipbs_url = 'https://www.thaipbs.or.th'
r = requests.get('https://www.thaipbs.or.th/search?q=%E0%B8%B7%E0%B8%B8%E0%B8%AA%E0%B9%88%E0%B8%87%E0%B8%AD%E0%B8%AD%E0%B8%81%E0%B8%97%E0%B8%B8%E0%B9%80%E0%B8%A3%E0%B8%B5%E0%B8%A2%E0%B8%99&siteType=news&type=&published_range=&page=1',)
b = soup(r.content, 'html.parser')

In [3]:
title = []
summary = []
content = []
published_datetime = []
retrive_datetime = []

In [ ]:
for news in b.findAll('div', attrs={'class' : 'ContentCardstyle__Container-sc-odesfa-0 iqGpPt'}):
    titles = news.find('p').get_text()
    #print(title)
    title.append(titles)
print(title)

In [5]:
url = []

for news in b.findAll('div', attrs={'class' : 'ContentInformationLayoutAstyle__Container-sc-1ut12ee-0 chsKqA'}):
    url.append(news.find('a', href=True)['href'])

In [ ]:
for link in url:
    page = requests.get(link)
    bsobj = soup(page.content, 'html.parser')
    for news in bsobj.findAll('h4'):
        s = news.text.strip()
        #print(news.text.strip())
        summary.append(s)
    
    for news in bsobj.findAll('div', attrs={'class':'Contentstyle__NewsContentContainer-sc-16duq30-0 hUUwiH'}):
        d = news.text
        di = d.strip()
        dis = di.replace('\n', '')
        disc = dis.replace('\xa0',' ')
        content.append(disc)

    for n in  bsobj.findAll('div', attrs={'class' : 'NewsHeaderstyle__MetaContainer-sc-1l82i9z-4 hWmbNM'}):
        te = n.find('span').get_text()
        published_datetime.append(te)
        
        retrive = str(datetime.date.today())
        retrive_datetime.append(retrive)
        # retrive_datetime.append(datetime.date.today())

print(retrive_datetime)
# print(summary)
# print(content)
# print(published_datetime)

In [ ]:
# col = ['Title','Publish Date','Content','Summary','Source_URL', 'Retrive_datetime']
table = pd.DataFrame([title,published_datetime,content,summary,url,retrive_datetime]).transpose()
table.columns = ['Title','Publish Date','Content','Summary','Source_URL','Retrive Date']

print(table)

In [31]:
table_csv = table.to_csv('data_thaipbs.csv', index=False)

In [32]:
table_1 = table.to_dict('records')

In [ ]:
table_1

In [ ]:
print(f"Total news : {len(table_1)}")

In [36]:
with open('data_thaipbs.json', 'w') as f:
    json.dump(table_1, f)

## Upload data to DataPlatform

In [37]:
from dsmlibrary.datanode import DataNode

In [38]:
dir_raw_id = 60

In [ ]:
datanode = DataNode()

In [ ]:
datanode.upload_file(directory_id=dir_raw_id, file_path='data_thaipbs.json')

In [ ]:
datanode.upload_file(directory_id=dir_raw_id, file_path='data_thaipbs.csv')